In [3]:
import pandas as pd
import numpy as np

r_cols = ['userId', 'movieId', 'rating']
ratings = pd.read_csv('../ml-latest-small/ratings.csv', sep=',', names=r_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)
#ratings = pd.read_csv('../ml-1m/ratings.dat', sep='::', names=r_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)

m_cols = ['movieId', 'title','genres']
movies = pd.read_csv('../ml-latest-small/movies.csv', sep=',', names=m_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)
#movies = pd.read_csv('../ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)

movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
ratings = pd.merge(movies, ratings)

ratings['rating'] = ratings['rating'].map(float)
userRatings = ratings.pivot_table(index=['title'],columns=['userId'],values='rating', aggfunc='first',fill_value=float('nan'))
userRatings.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$9.99 (2008),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Neath the Arizona Skies (1934),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
myUser = userRatings[643]
myUser.head()

title
"Great Performances" Cats (1998)          NaN
$9.99 (2008)                              NaN
'Hellboy': The Seeds of Creation (2004)   NaN
'Neath the Arizona Skies (1934)           NaN
'Round Midnight (1986)                    NaN
Name: 643, dtype: float64

In [9]:
corrMatrix = userRatings.corrwith(myUser)
corrMatrix = corrMatrix.sort_values(ascending = False)
corrMatrix.head(15)

userId
4      1.0
566    1.0
385    1.0
389    1.0
594    1.0
590    1.0
390    1.0
118    1.0
92     1.0
464    1.0
619    1.0
465    1.0
486    1.0
496    1.0
515    1.0
dtype: float64

In [52]:
#ratings.head()
corrList = pd.DataFrame(corrMatrix, columns=['x']).reset_index()
corrList.head()
#corrMatrix.head()
#ratings.head()

,userId,x
0,4,1.0
1,566,1.0
2,385,1.0
3,389,1.0
4,594,1.0


In [56]:
ratingsWithSimilarity = pd.merge(ratings, corrList, left_on = 'userId', right_on = 'userId')
ratingsWithSimilarity = ratingsWithSimilarity.dropna()
ratingsWithSimilarity['score'] = ratingsWithSimilarity['rating'] * ratingsWithSimilarity['x']
#ratingsWithSimilarity.sort_values(['score'], inplace= True)
ratingsWithSimilarity.head()

,movieId,title,genres,userId,rating,x,score
88,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",9,4.0,-1.0,-4.0
89,17,Sense and Sensibility (1995),"[Drama, Romance]",9,4.0,-1.0,-4.0
90,26,Othello (1995),[Drama],9,3.0,-1.0,-3.0
91,36,Dead Man Walking (1995),"[Crime, Drama]",9,5.0,-1.0,-5.0
92,47,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]",9,3.0,-1.0,-3.0


In [65]:
movieWithScore = ratingsWithSimilarity.groupby('title').sum()
recomendations = movieWithScore.sort_values('score', ascending = False).drop(myUser.dropna().index,errors = 'ignore')
recomendations.head(15)

,movieId,userId,rating,x,score
title,,,,,
Pulp Fiction (1994),54168,62435,793.5,77.093969,343.857270
Forrest Gump (1994),71200,67332,813.5,81.424326,328.954498
"Silence of the Lambs, The (1991)",100810,57874,704.5,65.917004,281.270003
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),179700,51071,641.5,57.378404,253.421835
Braveheart (1995),15730,46553,556.0,63.081336,252.341477
Terminator 2: Judgment Day (1991),87761,49169,606.0,60.867761,248.638435
Star Wars: Episode VI - Return of the Jedi (1983),169400,45350,583.0,56.085942,245.719421
American Beauty (1999),405836,46920,593.5,55.720906,235.611863
Jurassic Park (1993),73920,49886,567.0,62.031917,233.341749


In [68]:
myUser.dropna().sort_values(ascending = False)

title
Usual Suspects, The (1995)                               5.0
Star Wars: Episode IV - A New Hope (1977)                5.0
Fight Club (1999)                                        5.0
Godfather, The (1972)                                    5.0
Godfather: Part II, The (1974)                           5.0
Shawshank Redemption, The (1994)                         5.0
Dark Knight, The (2008)                                  5.0
Star Wars: Episode V - The Empire Strikes Back (1980)    5.0
Hobbit: An Unexpected Journey, The (2012)                4.5
Matrix, The (1999)                                       4.5
Jackal, The (1997)                                       3.5
U.S. Marshals (1998)                                     3.5
Ocean's Thirteen (2007)                                  3.0
Tango & Cash (1989)                                      3.0
Police Academy 4: Citizens on Patrol (1987)              2.5
Lethal Weapon 4 (1998)                                   2.5
Under Siege 2: Dar